In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np

def create_parametrized_circuit(params):
    """
    Implements a 2-to-1 DLOCCNet-style circuit with 8 parameters.
    Includes explicit Flag Logic (OR gate) for OpenQASM 3.0.
    """
    qr = QuantumRegister(4, 'q')
    cr = ClassicalRegister(3, 'c') # c[0]=Alice, c[1]=Bob, c[2]=Flag
    qc = QuantumCircuit(qr, cr)
    
    # --- 1. Parameterized Distillation Gates ---
    # Alice (q0, q1)
    qc.ry(params[0], qr[0])
    qc.ry(params[1], qr[1])
    qc.cx(qr[1], qr[0])      # CNOT Preserved -> Sacrificial
    
    qc.ry(params[3], qr[1])
    qc.cx(qr[0], qr[1])
    qc.ry(params[3], qr[1])

    # Bob (q3, q2) - Mirroring Alice
    qc.ry(params[4], qr[3])
    qc.ry(params[5], qr[2])
    qc.cx(qr[3], qr[2])      # CNOT Preserved -> Sacrificial
    qc.ry(params[6], qr[3])
    qc.cx(qr[2], qr[3])
    qc.ry(params[7], qr[3])

    # --- 2. Measurement ---
    qc.measure(qr[0], cr[0]) # Alice Outcome
    qc.measure(qr[3], cr[1]) # Bob Outcome

    # --- 3. Flag Logic (Implementing OR Gate) ---
    # We need c[2] = 1 if (c[0]=1 OR c[1]=1).
    # Since we can't write "c[2]=1" directly, we manipulate q0 and measure it into c[2].
    
    # Step A: Reset q0 to |0> using feedforward (Active Reset)
    with qc.if_test((cr[0], 1)):
        qc.x(qr[0])
        
    # Step B: Flip q0 to |1> if c[0] was 1
    with qc.if_test((cr[0], 1)):
        qc.x(qr[0])

    # Step C: Flip q0 to |1> if c[1] is 1, BUT only if it's currently |0>
    # Logic: if (c[1]==1 && c[0]==0) -> X q0
    with qc.if_test((cr[1], 1)):
        with qc.if_test((cr[0], 0)):
            qc.x(qr[0])
            
    # Step D: Measure Flag
    qc.measure(qr[0], cr[2])

    return qc

def optimize_8_params(client, edge_id, threshold):
    # 8 random initial parameters
    params = np.random.uniform(-0.1, 0.1, 8) 
    
    learning_rate = 0.1
    epsilon = 0.02
    max_iters = 30
    flag_bit = 2 # Server checks this bit. 0 = Success.
    
    print(f"Optimizing 8 parameters for edge {edge_id}...")

    for i in range(max_iters):
        grad = np.zeros(8)
        
        # Calculate Gradient for each parameter
        for j in range(8):
            # Perturb +epsilon
            p_plus = params.copy()
            p_plus[j] += epsilon
            qc_plus = create_parametrized_circuit(p_plus)
            
            # Claim with flag_bit=2 (The OR gate output)
            res_p = client.claim_edge(edge_id, qc_plus, flag_bit, 2)
            f_p = res_p["data"].get("fidelity", 0) if res_p.get("ok") else 0
            
            # Perturb -epsilon
            p_minus = params.copy()
            p_minus[j] -= epsilon
            qc_minus = create_parametrized_circuit(p_minus)
            
            res_m = client.claim_edge(edge_id, qc_minus, flag_bit, 2)
            f_m = res_m["data"].get("fidelity", 0) if res_m.get("ok") else 0
            
            # Gradient Ascent
            grad[j] = (f_p - f_m) / (2 * epsilon)
            
        # Update parameters
        params += learning_rate * grad
        params = np.clip(params, -np.pi, np.pi) # Keep params reasonable
        
        # Check progress (using last f_m as approx)
        print(f"Iter {i}: Approx Fidelity {f_m:.4f}")
        print(params)
        
        if f_m > threshold + 0.01:
            print("Threshold Met!")
            break
            
    return params